#### Digital Twin of a Food Oven ####

Rate of Energy In − Rate of Energy Out = Rate of Energy Stored

Governing Equation: The Foundation of Physics

Equation: $\frac{\delta T}{\delta t} = \alpha \frac{\delta^2 T}{\delta x^2}$

    What it is: This is the 1D Heat Equation (or Diffusion Equation). It's the mathematical rule that precisely describes how temperature (T) changes over time (t) and space (x) in a material.

- $\frac{\delta T}{\delta t}$: The rate of change of temperature at a specific point.

- $\frac{\delta^2T}{\delta x^2}$: The curvature of the temperature profile, representing how heat diffuses.

- α: The thermal diffusivity of the material (a constant indicating how quickly heat spreads).

Context: This is the true, high-fidelity physical law the system is supposed to obey. The PINN is designed to learn this complex solution

Lumped Capacitance Model :Physics Model → ODE: dT/dt = (u - T / R) / C

- Controller → PID
- PINN → Predicts T(t+1) from [T(t), u(t)]
- Visualization → Temp vs Setpoint

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import grad

In [ ]:
# PINN MODEL DEFINITION 

class OvenPINN(nn.Module):
    """
    Physics-Informed Neural Network for Oven Temperature Modeling.
    The network models the temperature T as a function of time (t) and
    one-dimensional space (x) within the oven.
    Input: (t, x) normalized between 0 and 1.
    Output: T (Temperature), normalized between 0 and 1.
    """
    def __init__(self, n_hidden=50, n_layers=4):
        super(OvenPINN, self).__init__()
        self.n_layers = n_layers

        # Input layer (t, x) -> 2 inputs
        self.input_layer = nn.Linear(2, n_hidden)

        # Hidden layers
        self.hidden_layers = nn.ModuleList([
            nn.Linear(n_hidden, n_hidden) for _ in range(n_layers)
        ])

        # Output layer (Temperature T) -> 1 output
        self.output_layer = nn.Linear(n_hidden, 1)

        # Activation function
        self.activation = nn.Tanh()

    def forward(self, x):
        """
        x is a tensor of shape (N, 2) where x[:, 0] is time t and x[:, 1] is position x.
        """
        out = self.activation(self.input_layer(x))
        for layer in self.hidden_layers:
            out = self.activation(layer(out))
        out = self.output_layer(out)
        return out

In [ ]:
# PHYSICS LOSS (Heat Equation) ---

def pde_loss(model, t_interior, x_interior, alpha):
    """
    Calculates the Physics Loss based on the 1D Heat Equation:
    ∂T/∂t = α * ∂²T/∂x²
    Loss = (∂T/∂t - α * ∂²T/∂x²)^2
    """
    # Combine t and x into a single tensor for input
    tx_interior = torch.cat([t_interior, x_interior], dim=1)

    # Enable gradients for inputs required for derivative calculation
    tx_interior.requires_grad_(True)

    # 1. Forward pass: T(t, x)
    T = model(tx_interior)

    # 2. Calculate first derivative: ∂T/∂t
    T_t = grad(T, tx_interior, torch.ones_like(T), create_graph=True)[0][:, 0:1]

    # 3. Calculate first derivative: ∂T/∂x
    T_x = grad(T, tx_interior, torch.ones_like(T), create_graph=True)[0][:, 1:2]

    # 4. Calculate second derivative: ∂²T/∂x²
    T_xx = grad(T_x, tx_interior, torch.ones_like(T_x), create_graph=True)[0][:, 1:2]

    # 5. Residual (Physics Loss): R = ∂T/∂t - α * ∂²T/∂x²
    # We want R to be close to 0
    pde_residual = T_t - alpha * T_xx

    # Mean Squared Error of the residual
    loss_pde = torch.mean(pde_residual ** 2)
    return loss_pde

In [ ]:
# CLASSIC PID CONTROLLER 

class PIDController:
    """
    Standard PID Controller implementation.
    Calculates the required heat input based on the error.
    """
    def __init__(self, Kp, Ki, Kd, output_limits=(0, 1000)):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.output_limits = output_limits
        self.P_term = 0
        self.I_term = 0
        self.D_term = 0
        self.last_error = 0
        self.integral_limit = 200 # Anti-windup limit

    def calculate(self, current_temp, setpoint, dt):
        error = setpoint - current_temp

        # Proportional Term
        self.P_term = self.Kp * error

        # Integral Term (with anti-windup)
        self.I_term += error * dt
        self.I_term = max(min(self.I_term, self.integral_limit), -self.integral_limit)
        I_output = self.Ki * self.I_term

        # Derivative Term
        if dt > 0:
            derivative = (error - self.last_error) / dt
            self.D_term = self.Kd * derivative
        else:
            self.D_term = 0

        # Controller Output (Heat Input, Qin)
        output = self.P_term + I_output + self.D_term

        # Clamp output to physical limits (e.g., 0 to 1000W)
        output = max(min(output, self.output_limits[1]), self.output_limits[0])

        # Update last error
        self.last_error = error

        return output

In [ ]:
# PHYSICS-BASED SIMULATION MODEL (Lumped Capacitance) ---

def simulate_physics_step(current_temp, Q_in, Tamb, dt, tau=500, heat_to_temp_factor=1e-3):
    """
    Simulates one step of the oven's heating using a simplified
    Lumped Capacitance/Newton's Law of Cooling model.

    dT/dt = (1/tau) * (Tamb - T) + Q_in * factor
    """
    # Heat loss term (Newton's Law of Cooling approximation)
    loss_rate = (Tamb - current_temp) / tau

    # Heater input term (Q_in is power, factor converts to temp rise rate)
    heat_rate = Q_in * heat_to_temp_factor

    dT = (loss_rate + heat_rate) * dt
    new_temp = current_temp + dT
    return new_temp

In [ ]:
# DATA GENERATION FOR TRAINING THE PINN ---

def generate_training_data(model_pinn, time_total, T_initial, T_setpoint, dt, pid_controller, sim_physics_step):
    """
    Runs a physics-based simulation to generate T(t) data points
    to train the PINN on actual dynamics.
    """
    print("Generating synthetic data for PINN training...")
    T_current = T_initial
    T_data = [T_initial]
    t_data = [0.0]
    t = 0.0
    steps = int(time_total / dt)

    # Use a dummy PID/Physics loop to generate realistic T(t) curve
    for _ in range(steps):
        Q_in = pid_controller.calculate(T_current, T_setpoint, dt)
        T_current = sim_physics_step(T_current, Q_in, 25.0, dt)
        t += dt
        T_data.append(T_current)
        t_data.append(t)

    # Convert to PyTorch tensors
    T_data = torch.tensor(T_data, dtype=torch.float32).unsqueeze(1)
    t_data = torch.tensor(t_data, dtype=torch.float32).unsqueeze(1)

    # Normalize data for PINN (assuming max temp is 250C)
    T_max = 250.0
    T_data_norm = T_data / T_max
    t_data_norm = t_data / time_total

    # Sample interior points for the PDE loss
    # Generate random points in (t, x) domain
    N_interior = 1000
    t_interior = torch.rand(N_interior, 1, requires_grad=True)
    x_interior = torch.rand(N_interior, 1, requires_grad=True)

    print(f"Data generated: {len(t_data)} points.")
    return t_data, T_data, T_data_norm, t_interior, x_interior, T_max, time_total

In [ ]:
# PINN TRAINING FUNCTION ---

def train_pinn(model_pinn, t_data, T_data_norm, t_interior, x_interior, T_initial_norm, T_setpoint_norm, time_total, T_max):
    """
    Trains the PINN using a combined loss: Data Loss + Physics Loss + IC/BC Loss.
    """
    optimizer = torch.optim.Adam(model_pinn.parameters(), lr=1e-3)
    loss_fn = nn.MSELoss()
    epochs = 5000 # Number of training epochs
    alpha = 0.01 # Thermal diffusivity constant (normalized for simplicity)

    # Combine t_data (normalized) with a fixed x=0.5 (middle of the oven)
    t_data_norm = t_data / time_total
    x_data = torch.ones_like(t_data_norm) * 0.5
    tx_data = torch.cat([t_data_norm, x_data], dim=1)

    # Initial Condition (IC) setup: T(t=0, x) = T_initial_norm
    t_ic = torch.zeros(100, 1) # 100 points at t=0
    x_ic = torch.rand(100, 1)  # distributed across space x
    tx_ic = torch.cat([t_ic, x_ic], dim=1)
    T_ic_target = torch.ones_like(t_ic) * T_initial_norm

    # Boundary Condition (BC) setup: Fixed T at boundaries x=0, x=1
    T_bc_target = torch.ones(100, 1) * T_setpoint_norm

    # Boundary x=0
    t_bc0 = torch.rand(50, 1)
    x_bc0 = torch.zeros(50, 1)
    tx_bc0 = torch.cat([t_bc0, x_bc0], dim=1)

    # Boundary x=1
    t_bc1 = torch.rand(50, 1)
    x_bc1 = torch.ones(50, 1)
    tx_bc1 = torch.cat([t_bc1, x_bc1], dim=1)
    
    # Combine all boundary points (x=0 and x=1) into one tensor (size 100)
    tx_bc = torch.cat([tx_bc0, tx_bc1], dim=0) 

    # Weights for the combined loss (Tuned for high data fidelity)
    w_data = 10.0  # Increased weight to prioritize the accurate data
    w_pde = 0.5 
    w_ic = 1.0
    w_bc = 0.5  # Lower weight to reduce conflict from simplified BC

    print("Starting PINN training...")
    for epoch in range(epochs):
        optimizer.zero_grad()

        # A. Data Loss (Supervised Learning) ---
        T_pred_data = model_pinn(tx_data)
        loss_data = loss_fn(T_pred_data, T_data_norm)

        # B. Physics Loss (PDE Residual) ---
        loss_pde = pde_loss(model_pinn, t_interior, x_interior, alpha)

        # C. Initial Condition (IC) Loss ---
        T_pred_ic = model_pinn(tx_ic)
        loss_ic = loss_fn(T_pred_ic, T_ic_target)

        # D. Boundary Condition (BC) Loss ---
        T_pred_bc = model_pinn(tx_bc)
        loss_bc = loss_fn(T_pred_bc, T_bc_target)

        # E. Total Loss ---
        total_loss = (w_data * loss_data) + (w_pde * loss_pde) + (w_ic * loss_ic) + (w_bc * loss_bc)

        total_loss.backward()
        optimizer.step()

        if (epoch + 1) % 500 == 0 or epoch == 0:
            print(f"Epoch {epoch+1}/{epochs} | Total Loss: {total_loss.item():.6f} | "
                  f"Data: {loss_data.item():.6f} | PDE: {loss_pde.item():.6f} | "
                  f"IC/BC: {(loss_ic + loss_bc).item():.6f}")

    print("PINN training complete.")
    return model_pinn

In [ ]:
# MAIN SIMULATION AND COMPARISON ---

def run_digital_twin():
    # --- Parameters ---
    T_INITIAL = 25.0      # Initial temperature (°C)
    T_SETPOINT = 180.0    # Target temperature (°C)
    T_AMBIENT = 25.0      # Ambient temperature (°C)
    SIM_TIME_S = 3600.0   # Total simulation time (1 hour in seconds)
    DT = 1.0              # Time step (1 second)

    # PID constants (tuned for the simple physics model)
    KP, KI, KD = 50.0, 0.01, 5.0 

    # Initialization ---
    pid = PIDController(Kp=KP, Ki=KI, Kd=KD, output_limits=(0, 1000))
    pinn_model = OvenPINN()

    # Data Generation & PINN Training ---
    # Generate data using the simplified physics model
    t_data, T_data, T_data_norm, t_interior, x_interior, T_max, time_total = \
        generate_training_data(pinn_model, SIM_TIME_S, T_INITIAL, T_SETPOINT, DT, pid, simulate_physics_step)

    # Reset PID for a fresh, identical simulation run
    pid = PIDController(Kp=KP, Ki=KI, Kd=KD, output_limits=(0, 1000))
    T_INITIAL_NORM = T_INITIAL / T_max
    # Calculate normalized setpoint to pass to the PINN trainer
    T_SETPOINT_NORM = T_SETPOINT / T_max

    # Train the PINN using the generated data and physics constraints
    pinn_model = train_pinn(pinn_model, t_data, T_data_norm, t_interior, x_interior, T_INITIAL_NORM, T_SETPOINT_NORM, time_total, T_max)
    pinn_model.eval() # Switch to evaluation mode

    # Digital Twin Simulation ---
    T_phys = T_INITIAL
    T_pinn = T_INITIAL
    time_series = [0.0]
    T_phys_series = [T_INITIAL]
    T_pinn_series = [T_INITIAL]
    T_setpoint_series = [T_SETPOINT]
    Q_in_series = [0.0]
    t = 0.0
    steps = int(SIM_TIME_S / DT)

    print("\nStarting Digital Twin Simulation (PID controlling Physics, PINN predicting)...")

    for _ in range(steps):
        # PID Controller determines heat input based on PHYSICS model's current temp
        Q_in = pid.calculate(T_phys, T_SETPOINT, DT)
        Q_in_series.append(Q_in)

        # Physics-Based Model (Digital Twin Core 1)
        T_phys = simulate_physics_step(T_phys, Q_in, T_AMBIENT, DT)

        # PINN Model Prediction (Digital Twin Core 2)
        t_norm = (t + DT) / SIM_TIME_S
        x_norm = 0.5 # Predict at the middle of the oven
        t_tensor = torch.tensor([[t_norm]], dtype=torch.float32)
        x_tensor = torch.tensor([[x_norm]], dtype=torch.float32)

        with torch.no_grad():
            T_pinn_norm = pinn_model(torch.cat([t_tensor, x_tensor], dim=1)).item()
            T_pinn = T_pinn_norm * T_max # Denormalize

        # Record Data
        t += DT
        time_series.append(t / 60.0) # Convert to minutes
        T_phys_series.append(T_phys)
        T_pinn_series.append(T_pinn)
        T_setpoint_series.append(T_SETPOINT)

    # Visualization ---
    plt.figure(figsize=(12, 7))
    plt.plot(time_series, T_setpoint_series, 'r--', label='Set Point (Target)')
    plt.plot(time_series, T_phys_series, 'b-', label='Physics Model (Ground Truth)')
    plt.plot(time_series, T_pinn_series, 'g:', linewidth=3, label='PINN Prediction')
    plt.title('Digital Twin Oven Temperature Control (PID + PINN Comparison)')
    plt.xlabel('Time (Minutes)')
    plt.ylabel('Temperature (°C)')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
if __name__ == '__main__':
    run_digital_twin()